In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
import math
import nltk

nltk.download('stopwords')

C:\Users\hp\AppData\Local\Temp\ipykernel_6576\1255534046.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("Musical_instruments_reviews.csv")
X = df['reviewText']
y = df['overall'].apply(lambda x: -1 if x <= 2 else (1 if x >= 4 else 0))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

stop_words = set(stopwords.words('english'))
preprocess = lambda text: ' '.join([word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in stop_words])
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

def calculate_tf(term, document):
    words = document.split()
    return words.count(term) / (len(words)+1)

def calculate_idf(term, documents):
    document_containing_term = sum(1 for document in documents if term in document.split())
    return math.log(len(documents) / (document_containing_term + 1)) if document_containing_term > 0 else 0

all_documents = X_train.tolist() + X_test.tolist()
idf_values = {term: calculate_idf(term, all_documents) for term in set(' '.join(all_documents).split())}

vocabulary = sorted(list(idf_values.keys()))

X_train_tfidf_manual = []
for document in X_train:
    tfidf_vector = [calculate_tf(term, document) * idf_values[term] for term in vocabulary]
    X_train_tfidf_manual.append(tfidf_vector)

X_test_tfidf_manual = []
for document in X_test:
    tfidf_vector = [calculate_tf(term, document) * idf_values[term] for term in vocabulary]
    X_test_tfidf_manual.append(tfidf_vector)

X_train_tfidf_manual = np.array(X_train_tfidf_manual)
X_test_tfidf_manual = np.array(X_test_tfidf_manual)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
model = LogisticRegression()
model.fit(X_train_tfidf_manual, y_train)
y_pred = model.predict(X_test_tfidf_manual)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.72      0.10      0.17       134
           0       0.53      0.89      0.66       229
           1       0.77      0.61      0.68       197

    accuracy                           0.60       560
   macro avg       0.67      0.53      0.51       560
weighted avg       0.66      0.60      0.55       560



In [8]:
new_text = "Worst Product"
new_text_tfidf_manual = [calculate_tf(term, preprocess(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:", "Positive" if predicted_sentiment[0] == 1 else "Neutral" if predicted_sentiment[0] == 0 else "Negative")

Predicted Sentiment: Negative


In [9]:
new_text = "ok product"
new_text_tfidf_manual = [calculate_tf(term, preprocess(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:", "Positive" if predicted_sentiment[0] == 1 else "Neutral" if predicted_sentiment[0] == 0 else "Negative")

Predicted Sentiment: Neutral


In [13]:
new_text = "amazing product and highly recommended"
new_text_tfidf_manual = [calculate_tf(term, preprocess(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:", "Positive" if predicted_sentiment[0] == 1 else "Neutral" if predicted_sentiment[0] == 0 else "Negative")

Predicted Sentiment: Positive
